In [2]:
import sys

sys.path.append('C:/MASTIXStudio/Python')

import numpy as np
import pandas as pd
from mastixpy import Value

Read cash flow (200k + 200k), discount factor and fx rate files.

In [3]:
%%time

sekCashFlows = pd.read_csv('C:/Users/JensSvensson/OneDrive - Mastix/Work/Batch Examples/CSV_400k/SEK_CashFlows_20220816.csv', delimiter=';', decimal=',', parse_dates=[3])
nokCashFlows = pd.read_csv('C:/Users/JensSvensson/OneDrive - Mastix/Work/Batch Examples/CSV_400k/NOK_CashFlows_20220816.csv', delimiter=';', decimal=',', parse_dates=[3])

sekDiscountFactors = pd.read_csv('C:/Users/JensSvensson/OneDrive - Mastix/Work/Batch Examples/CSV_400k/SEK_Discount_Factors_20220816.csv', delimiter=';', decimal=',', index_col=0, parse_dates=[0])
nokDiscountFactors = pd.read_csv('C:/Users/JensSvensson/OneDrive - Mastix/Work/Batch Examples/CSV_400k/NOK_Discount_Factors_20220816.csv', delimiter=';', decimal=',', index_col=0, parse_dates=[0])

fxRates = pd.read_csv('C:/Users/JensSvensson/OneDrive - Mastix/Work/Batch Examples/CSV_400k/Fx_Rates_20220816.csv', delimiter=';', decimal=',', index_col=0, usecols=lambda x: x != 'Price Currency')

# Possibly drop zero derivative columns.

Wall time: 5.5 s


Change column names to better be in line with intention.

In [4]:
sekCashFlows.rename(columns={'Amount': 'Forward Value (Currency)'}, inplace=True)
nokCashFlows.rename(columns={'Amount': 'Forward Value (Currency)'}, inplace=True)

We can choose to have derivative columns ordered to taste.

In [5]:
preferred_derivatives_order = ['SEK Appreciation', 'NOK Appreciation', 'USD Appreciation',
                               'USDNOK Fx Spot Rate', 'USDSEK Fx Spot Rate',
                               'SEK.DSC.1D', 'SEK.DSC.1W', 'SEK.DSC.2W', 'SEK.DSC.1M', 'SEK.DSC.2M',
                               'SEK.DSC.3M', 'SEK.DSC.6M', 'SEK.DSC.1Y', 'SEK.DSC.2Y', 'SEK.DSC.3Y',
                               'SEK.DSC.5Y', 'SEK.DSC.10Y', 'SEK.DSC.15Y', 'SEK.DSC.20Y', 'SEK.DSC.30Y',
                               'NOK.DSC.1D', 'NOK.DSC.1W', 'NOK.DSC.2W', 'NOK.DSC.1M', 'NOK.DSC.2M',
                               'NOK.DSC.3M', 'NOK.DSC.6M', 'NOK.DSC.1Y', 'NOK.DSC.2Y', 'NOK.DSC.3Y',
                               'NOK.DSC.5Y', 'NOK.DSC.10Y', 'NOK.DSC.15Y', 'NOK.DSC.20Y', 'NOK.DSC.30Y'
                               'USD.DSC.1D', 'USD.DSC.1M', 'USD.DSC.1W', 'USD.DSC.2W',
                               'SEK.3M.1D', 'SEK.3M.1W', 'SEK.3M.2W', 'SEK.3M.1M', 'SEK.3M.2M',
                               'SEK.3M.3M', 'SEK.3M.6M', 'SEK.3M.1Y', 'SEK.3M.2Y', 'SEK.3M.3Y',
                               'SEK.3M.5Y', 'SEK.3M.10Y', 'SEK.3M.15Y', 'SEK.3M.20Y', 'SEK.3M.30Y',
                               'NOK.3M.1D', 'NOK.3M.1W', 'NOK.3M.2W', 'NOK.3M.1M', 'NOK.3M.2M',
                               'NOK.3M.3M', 'NOK.3M.6M', 'NOK.3M.1Y', 'NOK.3M.2Y', 'NOK.3M.3Y',
                               'NOK.3M.5Y', 'NOK.3M.10Y', 'NOK.3M.15Y', 'NOK.3M.20Y', 'NOK.3M.30Y']

Bind cash flows to discount factors and fx rates.

In [6]:
%%time

sek_cf_df = Value(sekCashFlows[['Pay']].merge(sekDiscountFactors, left_on='Pay', right_index=True), 'Discount Factor', sekDiscountFactors.columns[1:])
nok_cf_df = Value(nokCashFlows[['Pay']].merge(nokDiscountFactors, left_on='Pay', right_index=True), 'Discount Factor', nokDiscountFactors.columns[1:])

sek_cf_fx = Value(sekCashFlows[['Currency']].merge(fxRates, left_on='Currency', right_index=True), 'Fx Rate', fxRates.columns[1:])
nok_cf_fx = Value(nokCashFlows[['Currency']].merge(fxRates, left_on='Currency', right_index=True), 'Fx Rate', fxRates.columns[1:])

sek_cf_df_SEK = Value(sekCashFlows[['Pay']].merge(sekDiscountFactors, left_on='Pay', right_index=True), 'Discount Factor', sekDiscountFactors.columns[1:])
nok_cf_df_SEK = Value(nokCashFlows[['Pay']].merge(sekDiscountFactors, left_on='Pay', right_index=True), 'Discount Factor', sekDiscountFactors.columns[1:])

Wall time: 2.73 s


Calculate present value in currency as well as present value and forward value in SEK together with derivatives (sensitivities) against all risk factors.

\begin{align*}
fv^{c} &= a^{c} \\
pv^{c} &= fv^{c} df^{c} \\
pv^{SEK} &= pv^{c} fx^{c,SEK} \\
fv^{SEK} &= \frac{pv^{SEK}}{df^{SEK}} \\
\end{align*}

\begin{align*}
\frac{dfv^{c}}{dx} &= \frac{da^{c}}{dx} \\
\frac{dpv^{c}}{dx} &= \frac{dfv^{c}}{dx} df^{c} + fv^{c} \frac{ddf^{c}}{dx} \\
\frac{dpv^{SEK}}{dx} &= \frac{dpv^{c}}{dx} fx^{c,SEK}  + pv^{c} \frac{dfx^{c,SEK}}{dx} \\
\frac{dfv^{SEK}}{dx} &= \frac{dpv^{SEK}}{dx} \frac{1}{df^{SEK}} - pv^{SEK} \frac{1}{(df^{SEK})^2} \frac{ddf^{SEK}}{dx} 
\end{align*}

In [7]:
%%time

sek_cf_fv = Value(sekCashFlows, 'Forward Value (Currency)', sekCashFlows.columns[6:], ['Trade Id', 'Cash Flow', 'Pay'], preferred_derivatives_order)
nok_cf_fv = Value(nokCashFlows, 'Forward Value (Currency)', nokCashFlows.columns[6:], ['Trade Id', 'Cash Flow', 'Pay'], preferred_derivatives_order)

sek_cf_pv = sek_cf_fv.multiply(sek_cf_df, 'Present Value (Currency)', preferred_derivatives_order)
nok_cf_pv = nok_cf_fv.multiply(nok_cf_df, 'Present Value (Currency)', preferred_derivatives_order)

sek_cf_pv_SEK = sek_cf_pv.multiply(sek_cf_fx, 'Present Value (SEK)', preferred_derivatives_order)
nok_cf_pv_SEK = nok_cf_pv.multiply(nok_cf_fx, 'Present Value (SEK)', preferred_derivatives_order)

sek_cf_fv_SEK = sek_cf_pv_SEK.divide(sek_cf_df_SEK, 'Forward Value (SEK)', preferred_derivatives_order)
nok_cf_fv_SEK = nok_cf_pv_SEK.divide(nok_cf_df_SEK, 'Forward Value (SEK)', preferred_derivatives_order)

Wall time: 9.62 s


For convenience - set index based on Trade Id and Cash flow number (within trade).

In [8]:
%%time

sek_cf_fv = sek_cf_fv.data_frame.set_index(['Trade Id', 'Cash Flow'])
nok_cf_fv = nok_cf_fv.data_frame.set_index(['Trade Id', 'Cash Flow'])

sek_cf_pv = sek_cf_pv.data_frame.set_index(['Trade Id', 'Cash Flow'])
nok_cf_pv = nok_cf_pv.data_frame.set_index(['Trade Id', 'Cash Flow'])

sek_cf_pv_SEK = sek_cf_pv_SEK.data_frame.set_index(['Trade Id', 'Cash Flow'])
nok_cf_pv_SEK = nok_cf_pv_SEK.data_frame.set_index(['Trade Id', 'Cash Flow'])

sek_cf_fv_SEK = sek_cf_fv_SEK.data_frame.set_index(['Trade Id', 'Cash Flow'])
nok_cf_fv_SEK = nok_cf_fv_SEK.data_frame.set_index(['Trade Id', 'Cash Flow'])

Wall time: 3.69 s


Calculate memory usage.

In [9]:
print(f'sek_cf_fv: {sek_cf_fv.memory_usage().sum()/1e6:,} mb')
print(f'nok_cf_fv: {nok_cf_fv.memory_usage().sum()/1e6:,} mb')
print()
print(f'sek_cf_pv: {sek_cf_pv.memory_usage().sum()/1e6:,} mb')
print(f'nok_cf_pv: {nok_cf_pv.memory_usage().sum()/1e6:,} mb')
print()
print(f'sek_cf_pv_SEK: {sek_cf_pv_SEK.memory_usage().sum()/1e6:,} mb')
print(f'nok_cf_pv_SEK: {nok_cf_pv_SEK.memory_usage().sum()/1e6:,} mb')
print()
print(f'sek_cf_fv_SEK: {sek_cf_fv_SEK.memory_usage().sum()/1e6:,} mb')
print(f'nok_cf_fv_SEK: {nok_cf_fv_SEK.memory_usage().sum()/1e6:,} mb')

sek_cf_fv: 298.988894 mb
nok_cf_fv: 298.058656 mb

sek_cf_pv: 548.486054 mb
nok_cf_pv: 530.184592 mb

sek_cf_pv_SEK: 748.083782 mb
nok_cf_pv_SEK: 729.14968 mb

sek_cf_fv_SEK: 748.083782 mb
nok_cf_fv_SEK: 911.534344 mb


Now, we are in a position where we, for example, have the forward value in SEK for every single cash flow in every loan with derivatives against all relevant risk factors. We can also bring along some additional columns to taste - in excess to the calculated value and its derivatives - in this case the cash flow pay date. By using for example, the groupby and slicing functionality in pandas we can very easily gain insights into calculated values as well as what drive its changes.

In [10]:
nok_cf_fv_SEK

Pay  Forward Value (SEK)  SEK Appreciation  \
Trade Id Cash Flow                                                     
A1       1         2022-08-25          1200.756457      -1200.756457   
         2         2022-09-26          1239.490536      -1239.490536   
         3         2022-10-25          1123.288299      -1123.288299   
         4         2022-11-25          1200.756457      -1200.756457   
         5         2022-12-27          1239.490536      -1239.490536   
...                       ...                  ...               ...   
C100000  1         2022-08-29           121.878553       -121.878553   
         2         2022-09-27           150.263413       -150.263413   
         3         2022-10-27           158.038952       -158.038952   
         4         2022-11-28           171.242295       -171.242295   
         5         2022-11-28        114127.132215    -114127.132215   

                    NOK Appreciation  USD Appreciation  USDNOK Fx Spot Rate  \
Trade Id Cash Flow                                                            
A1       1               1200.756457     -2.459910e-13          -147.718142   
         2               1239.490536     -2.539262e-13          -152.483243   
         3               1123.288299     -2.301206e-13          -138.187939   
         4               1200.756457     -2.459910e-13          -147.718142   
         5               1239.490536     -2.539262e-13          -152.483243   
...                              ...               ...                  ...   
C100000  1                121.878553     -2.496845e-14           -14.993609   
         2                150.263413     -3.078347e-14           -18.485540   
         3                158.038952     -3.237639e-14           -19.442094   
         4                171.242295     -3.508128e-14           -21.066381   
         5             114127.132215     -2.338047e-11        -14040.022665   

                    USDSEK Fx Spot Rate  SEK.DSC.1D  SEK.DSC.1W  SEK.DSC.2W  \
Trade Id Cash Flow                                                            
A1       1                   136.297811   -5.512703   20.158712    8.504502   
         2                   140.694515   -5.690532   20.808993    8.778841   
         3                   127.504404   -5.157044   18.858150    7.955824   
         4                   136.297811   -5.512703   20.158712    8.504502   
         5                   140.694515   -5.690532   20.808993    8.778841   
...                                 ...         ...         ...         ...   
C100000  1                    13.834429   -0.559547   -0.111910    0.000000   
         2                    17.056393   -0.689863   -0.137973    0.000000   
         3                    17.938994   -0.725561   -0.145112    0.000000   
         4                    19.437705   -0.786178   -0.157236    0.000000   
         5                 12954.565620 -523.960495 -104.792141    0.000000   

                    ...    NOK.3M.3M     NOK.3M.6M  NOK.3M.1Y  NOK.3M.2Y  \
Trade Id Cash Flow  ...                                                    
A1       1          ...     0.000000      0.000000        0.0        0.0   
         2          ...     0.000000      0.000000        0.0        0.0   
         3          ...     0.000000      0.000000        0.0        0.0   
         4          ...     0.000000      0.000000        0.0        0.0   
         5          ...     0.000000      0.000000        0.0        0.0   
...                 ...          ...           ...        ...        ...   
C100000  1          ...     0.000000      0.000000        0.0        0.0   
         2          ...  8829.703110   1452.989119        0.0        0.0   
         3          ...  7550.548881   6070.049101        0.0        0.0   
         4          ...  1088.167436  13838.603431        0.0        0.0   
         5          ...     0.000000      0.000000        0.0        0.0   

                    NOK.3M.3Y  NOK.3M.5Y  NO